# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости.

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [ ]:
!pip install lightgbm

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
try:
    data = pd.read_csv('/Users/pashc/Downloads/autos.csv')
except:
    data = pd.read_csv('/datasets/autos.csv')

In [ ]:
data.head()

In [ ]:
data.info(),data.isna().sum()

Мы видим, что есть пропуски в некоторых столбцах.Для исключения выбросов заменим

In [ ]:
data["VehicleType"] = data["VehicleType"].fillna(data["VehicleType"].mode().values[0])
data["Gearbox"] = data["Gearbox"].fillna(data["Gearbox"].mode().values[0])
data["Model"] = data["Model"].fillna(data["Model"].mode().values[0])
data["FuelType"] = data["FuelType"].fillna(data["FuelType"].mode().values[0])
data["Repaired"] = data["Repaired"].fillna(data["Repaired"].mode().values[0])

data.dropna(subset=['model'], inplace=True)
data.dropna(subset=['gearbox'], inplace=True)
data.dropna(subset=["vehicletype"], inplace=True)
data.dropna(subset=["fueltype"], inplace=True)
data.dropna(subset=["repaired"], inplace=True)

Приведем к нижнему регистру

In [ ]:
data.columns = data.columns.str.lower()

Смотрим, если дубликаты

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)

Удалим столбцы, которые нам не нужны в дальнейшей работе

In [ ]:
data = data.drop(["datecrawled","datecreated","lastseen","numberofpictures","postalcode"],axis = 1)

In [ ]:
data.head()

In [ ]:
data.price.unique()

Проанализируем детально целевой признак

In [ ]:
sns.set_style('darkgrid')
data.price.hist(bins=50,figsize=(15,10))
plt.title('Распределение стоимости автомобилей')
plt.xlabel('Стоимость')
plt.ylabel('Количество')
plt.show()

Можно заметить аномалию- мишины в нормальном состоянии не могут столько стоить. Поэтому удалим данные стоимости автомобилей ниже 1000

In [ ]:
data = data.query('price> 10')

In [ ]:
data.vehicletype.unique()

In [ ]:
data.registrationyear.unique()

Выявлена аномалия, поэтому не будем включать в данные значение до 1900 и после 2023

In [ ]:
data = data.loc[(data['registrationyear'] <= 2023) & (data['registrationyear'] >= 1900)]

In [ ]:
data.gearbox.unique()

In [ ]:
data.power.unique()

Выявлена аномалия, поэтому не будем включать в данные силы 700

In [ ]:
data = data.loc[data['power'] <= 700]

In [ ]:
data.model.unique()

In [ ]:
data.kilometer.unique()

In [ ]:
data.registrationmonth.unique()

In [ ]:
data.fueltype.unique()

In [ ]:
data.brand.unique()

In [ ]:
data.repaired.unique()

In [ ]:
data.duplicated().sum()

**Вывод**.
- В данных были обнаружены пропущенные значения и удалены. Более того, мы обнаружили и удалили дубликаты, исключили из данных столбцы, которые не несут важной информации И обработали аномалии в столбцах.

## Обучение моделей

In [ ]:
features=data.drop(['price'],axis=1)
target=data['price']

In [ ]:
col=['vehicletype', 'gearbox','model',
       'fueltype', 'brand', 'repaired']

Поделим данные на выборки

In [ ]:
features_train, features_valid_test, target_train, target_valid_test  = train_test_split(features,target, test_size=0.4, random_state=254)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

features_train.shape, features_valid.shape, features_test.shape

Скопируем данные для кодировки

In [ ]:
features_train_oe = features_train.copy()
features_valid_oe = features_valid.copy()
features_test_oe = features_test.copy()

Приступаем к кодировке

In [ ]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encoder.fit(features_train[col])
features_train_oe[col] = encoder.transform(features_train_oe[col])
features_valid_oe[col] = encoder.transform(features_valid_oe[col])
features_test_oe[col] = encoder.transform(features_test_oe[col])
features_test_oe.head()

<div class="alert alert-block alert-success">
<b>Успех:</b> OE - хороший выбор
</div>


In [ ]:
for depth in range(3, 20, 3):
    model = DecisionTreeRegressor(random_state=12345, max_depth = depth)
    model.fit(features_train_oe, target_train)
    predictions = model.predict(features_valid_oe)
    rmse = mean_squared_error(target_valid, predictions)**0.5
    print('Глубина:', depth)
    print('RMSE для решающего дерева:', rmse)
    print('')

Лучший результат модели решающего дерева - глубина: 12, rmse: 1974

In [ ]:
for depth in range(1,10):
    for est in range(5,20,5):
        model = RandomForestRegressor(max_depth=depth, n_estimators=est, random_state=1234)
        model.fit(features_train_oe, target_train)
        predictions = model.predict(features_valid_oe)
        rmse = mean_squared_error(target_valid, predictions)**0.5
        print('Количество деревьев:', est, "Глубина",depth)
        print('RMSE для случайного леса:', rmse)
        print('')

Лучший результат модели случайного леса - глубина: 9, rmse: 2028

In [ ]:
for est in [100, 500, 1000]:
    for num in [25, 400, 15]:
        model = lgb.LGBMRegressor(n_estimators = est, num_leaves = num, random_state=12345)
        model.fit(features_train_oe, target_train)
        predictions= model.predict(features_valid_oe)
        rmse = mean_squared_error(target_valid, predictions)**0.5
        print('Количество деревьев:', est, num)
        print('RMSE для случайного леса:', rmse)
        print('')

Лучший результат модели - rmse: 1588

**Вывод**
- лучше всего себя показала модель LightGBM, с результатом 1588.

## Анализ моделей

In [ ]:
%%time
model = DecisionTreeRegressor(random_state=12345, max_depth=15)
model.fit(features_train_oe, target_train)

In [ ]:
%%time
predictions = model.predict(features_valid_oe)
rmse = mean_squared_error(target_valid, predictions)**0.5
print(rmse)

In [ ]:
%%time
model1 = RandomForestRegressor(random_state=12345, max_depth=9, n_estimators=15)
model1.fit(features_train_oe, target_train)

In [ ]:
%%time
predictions = model1.predict(features_valid_oe)
rmse = mean_squared_error(target_valid, predictions)**0.5
print(rmse)

In [ ]:
%%time
model2 = lgb.LGBMRegressor(random_state=12345,
                          n_estimators=500,
                          num_leaves=400)
model2.fit(features_train_oe, target_train)

In [ ]:
%%time
predictions = model2.predict(features_valid_oe)
rmse = mean_squared_error(target_valid, predictions)**0.5
print(rmse)

**Проверка лучшей модели**

По резульататм качества предсказания, времени обучения модели и времени предсказания модели была выбрана мрдель решаещего дерева

In [ ]:
%%time
predictions = model.predict(features_test_oe)
rmse = mean_squared_error(target_test, predictions)**0.5
print(rmse)

**Вывод**
- результат модели : rmse меньше 2500, со временем 15.3 ms